# OCR using CNN

Importing the libraries

In [125]:
from keras.layers import Dense, Dropout, Convolution2D, Activation, MaxPooling2D, Layer, AveragePooling2D
from keras.layers import Flatten
from keras.models import Sequential
import scipy
import numpy as np
import os
import keras.utils
import tensorflow as tf
from keras import backend as K

Loading data form EMNIST database

In [122]:
EMNIST = scipy.io.loadmat(os.path.join(os.getcwd(), "matlab", "emnist-digits.mat"))

x_train = EMNIST["dataset"][0][0][0][0][0][0].astype("float64")
y_train = EMNIST["dataset"][0][0][0][0][0][1]

x_test = EMNIST['dataset'][0][0][1][0][0][0].astype("float64")
y_test = EMNIST['dataset'][0][0][1][0][0][1]

# Scaling data
x_train = (x_train - np.mean(x_train)) /np.std(x_train)
x_test =  (x_test - np.mean(x_train)) /np.std(x_train)

nb_classes = 10

y_train = keras.utils.to_categorical(y_train, nb_classes)
y_test = keras.utils.to_categorical(y_test, nb_classes)

print("Fraction test {}, Fraction train {}".format(len(y_test)/(len(y_test) + len(y_train)), len(y_train)/(len(y_test) + len(y_train))))

Fraction test 0.14285714285714285, Fraction train 0.8571428571428571


In [123]:
x_train_scaled = x_train.reshape(-1, 28, 28, 1).astype('float32')
x_test_scaled = x_test.reshape(-1, 28, 28, 1).astype('float32')

x_train_scaled = tf.pad(tensor = x_train_scaled, paddings=[[0, 0], [2, 2], [2, 2], [0, 0]])
x_test_scaled = tf.pad(tensor = x_test_scaled, paddings=[[0, 0], [2, 2], [2, 2], [0, 0]])

In [126]:
height = 32
width = 32
input_shape = (height, width, 1)


pool_size = (2, 2) # size of pooling area for max pooling
kernel_size = (3, 3) # convolution kernel size

# model = keras.Sequential([
#     keras.layers.Conv2D(6, (5, 5), activation='tanh', input_shape=(32, 32, 1)),
#     keras.layers.AveragePooling2D((2, 2)),
#     keras.layers.Conv2D(16, (5, 5), activation='tanh'),
#     keras.layers.AveragePooling2D((2, 2)),
#     keras.layers.Flatten(),
#     keras.layers.Dense(120, activation='tanh'),
#     keras.layers.Dense(84, activation='tanh'),
#     keras.layers.Dense(27, activation='softmax'),
# ])

# model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=['accuracy'])
# print(model.summary())
# history = model.fit(x_train_scaled, y_train, epochs=10, validation_split=0.1)


model=Sequential()
# C1
model.add(Convolution2D(6, (5, 5),
                        padding='valid',
                        input_shape=input_shape,
                        activation='tanh'))

# S2
model.add(AveragePooling2D((2,2)))

#C3
model.add(Convolution2D(16,
                        (5, 5),
                        padding='valid',
                        activation='tanh'))
model.add(Dropout(0.25))
# S4
model.add(AveragePooling2D(pool_size=(2, 2), padding='valid'))

model.add(Flatten())

# C5
model.add(Dense(120, activation='tanh'))

model.add(Dense(84, activation='tanh'))
# F6
model.add(Dense(nb_classes, activation='softmax'))
# model.summary()

model.compile(loss="categorical_crossentropy", optimizer='SGD', metrics=['accuracy'])


# # model.compile(loss = ///,
# #               optimizer = ///,
# #               metrics = ["accuracy"]
# #               )
batch = 100
epoch = 10
model.fit(x_train_scaled, y_train,
            batch_size=batch,
            epochs=epoch,
            verbose=1,
            validation_data=(x_test_scaled, y_test))

# score = model.evaluate(x_test_scaled, y_test, verbose=0)


c:\Users\msii\lepl1109\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 45s 18ms/step - accuracy: 0.8283 - loss: 0.6599 - val_accuracy: 0.5016 - val_loss: 1.6977
Epoch 2/10
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 43s 18ms/step - accuracy: 0.9489 - loss: 0.1802 - val_accuracy: 0.6491 - val_loss: 1.2396
Epoch 3/10
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 41s 17ms/step - accuracy: 0.9645 - loss: 0.1252 - val_accuracy: 0.7197 - val_loss: 0.9443
Epoch 4/10
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 67s 28ms/step - accuracy: 0.9725 - loss: 0.0959 - val_accuracy: 0.7579 - val_loss: 0.8097
Epoch 5/10
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 52s 22ms/step - accuracy: 0.9770 - loss: 0.0797 - val_accuracy: 0.7792 - val_loss: 0.7431
Epoch 6/10
2400/2400 ━━━━━━━━━━━━━━━━━━━━ 49s 20ms/step - accuracy: 0.9799 - loss: 0.0707 - val_accuracy: 0.7962 - val_loss: 0.6719
Epoch 7/10
1398/2400 ━━━━━━━━━━━━━━━━━━━━ 23s 23ms/step - accuracy: 0.9811 - loss: 0.0639

KeyboardInterrupt: 

In [ ]:
y_pred = model.predict(x_train_scaled[1].reshape(1, height, width, 1))

print(y_train[1])
print(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[[4.0305740e-06 7.2966868e-06 4.2318193e-06 3.5341807e-05 3.8730315e-04
  3.2485841e-04 6.3729985e-08 1.1710874e-02 2.0888053e-02 9.6663797e-01]]
